Make API call of Guardian to return all Jay Rayner articles. First query the API for the list of articles - this returns a page of results. We fetch the first batch of links to follow here, then we need to work through the rest of the list of pages, getting all those links.  

In [ ]:
import requests
import re 
import os

page_size = 15
page = 1

#we need to fail if the key isnt set
try:
    api_key = os.environ['GUARDIAN_API_KEY']
except:
    raise SystemExit("specify the API key as an env var before continuing")


#base url which looks for the jay tags 
url = f"https://content.guardianapis.com/search?api-key={api_key}&tag=food/series/jay-rayner-on-restaurants&page-size={page_size}"
payload = {}
headers = {
  'Cookie': 'AWSELB=75B9BD811C5C032EDEF76366759629DCCB8726D7A371904BEC1C3B7DFC40019571E370E2C4E4519DDF3CD336789F71716B110728D88A7C69AE901D39C1821FF2C5E227F5F9; AWSELBCORS=75B9BD811C5C032EDEF76366759629DCCB8726D7A371904BEC1C3B7DFC40019571E370E2C4E4519DDF3CD336789F71716B110728D88A7C69AE901D39C1821FF2C5E227F5F9; BCSI-CS-682e9ee7ab0fdcff=1'
}

#function to go through the results
def review_grabber(apiresults, count):
    #we need to open two files to save the results in, then we need to send them to the gsheet as separate layers
    with open('jaymap_pcode.tsv', 'a') as pcode_tsv_file, open("jaymap_nocode.tsv", 'a') as nocode_tsv_file:
        #if status is ok, work through the list of results and find the postcodes in the linked articles
        if apiresults['response']['status'] == 'ok':
            for result in apiresults['response']['results']:
                #print(f"visiting {result['webUrl']}, {count}, {len(apiresults)}")
                article = requests.request("GET", result['webUrl'], headers=headers, data = payload)
                postcode = re.findall(r'\b[A-Z]{1,2}[0-9][A-Z0-9]? [0-9][ABD-HJLNP-UW-Z]{2}\b', article.text)
                #if there is a postcode print it, to avoid out of bounds error caught in if loop
                if len(postcode) > 0:
                    #output as TSV as there are commas in some of the titles
                    pcode_tsv_file.write(f"\n{result['webTitle']}\t{result['webUrl']}\t{postcode[0]}\t{result['webPublicationDate']}")
                    print(count, result['webTitle'], result['webUrl'], postcode[0], result['webPublicationDate'])
                else:
                    nocode_tsv_file.write(f"\n{result['webTitle']}\t{result['webUrl']}\t{result['webPublicationDate']}")
                    print(count, result['webTitle'], result['webUrl'], result['webPublicationDate'])
                count += 1 
            return count

#fetch the first page listing articles by jay
response = requests.request("GET", url, headers=headers, data = payload)
responsejson = response.json()

pages = responsejson['response']['pages']
#total articles to check off later
total_articles = responsejson['response']['total']
print(f"there are {pages} pages of articles to get, {page_size} articles on each page, {total_articles} articles in total")
#send the whole response into the function to visit the page and get the postcode
count = 0 
count = review_grabber(responsejson, count)

In [ ]:
#work through the rest of the pages getting the links
page = 2
while page <= pages:
    print(page)
    url = f"https://content.guardianapis.com/search?api-key={api_key}&tag=food/series/jay-rayner-on-restaurants&page-size={page_size}&page={page}"
    response2 = requests.request("GET", url, headers=headers, data = payload)
    response2json = response2.json()
    count = review_grabber(response2json, count)
    page += 1

In [ ]:
page_size = 5
page = 2
api_key = os.environ.get('GUARDIAN_API_KEY')

#base url which looks for the jay tags 
url = f"https://content.guardianapis.com/search?api-key={api_key}&tag=food/series/jay-rayner-on-restaurants&page-size={page_size}&page={page}"
payload = {}
headers = {
  'Cookie': 'AWSELB=75B9BD811C5C032EDEF76366759629DCCB8726D7A371904BEC1C3B7DFC40019571E370E2C4E4519DDF3CD336789F71716B110728D88A7C69AE901D39C1821FF2C5E227F5F9; AWSELBCORS=75B9BD811C5C032EDEF76366759629DCCB8726D7A371904BEC1C3B7DFC40019571E370E2C4E4519DDF3CD336789F71716B110728D88A7C69AE901D39C1821FF2C5E227F5F9; BCSI-CS-682e9ee7ab0fdcff=1'
}
count = 1
#function to go through the results
def review_grabber(apiresults, count):
    with open('jaymap_new.csv', 'a') as csvfile:
        #if status is ok, work through the list of results and find the postcodes in the linked articles
        if apiresults['response']['status'] == 'ok':
            for result in apiresults['response']['results']:
                article = requests.request("GET", result['webUrl'], headers=headers, data = payload)
                postcode = re.findall(r'\b[A-Z]{1,2}[0-9][A-Z0-9]? [0-9][ABD-HJLNP-UW-Z]{2}\b', article.text)
                if len(postcode) > 0:
                    csvfile.write(f"\n{count},{result['webTitle']},{result['webUrl']},{postcode[0]}")
                    print(count, result['webTitle'], result['webUrl'], postcode[0])
                count += 1
            return count

#fetch the first page listing articles by jay
response = requests.request("GET", url, headers=headers, data = payload)
responsejson = response.json()
responsejson

In [ ]:

#how many pages are there in the response, as we will need to get them all
pages = responsejson['response']['pages']
#total articles to check off later
total_articles = responsejson['response']['total']
print(f"there are {pages} pages of articles to get, {page_size} articles on each page, {total_articles} articles in total")
#send the whole response into the function to visit the page and get the postcode